# Python

## O duplo significado de uma linguagem interpretada

Que Python é uma linguagem de programação, já sabemos. Enquanto linguagem, Python possui um conjunto bem-definido de regras: léxicas, sintáticas, semânticas e de lógica abstrata.

Porém, algo que em cursos introdutórios por vezes é dito, mas nem sempre explorado, é que Python é uma linguagem interpretada. Isso significa que Python é mais que apenas uma linguagem: ele é também um programa. Inclusive, usaremos aqui a palavra Python com esse duplo significado. Quando clicamos no executável chamado ```python3``` no computador, ou o executamos do terminal, ou ainda deixamos uma IDE ou interface executá-lo por nós, estamos rodando tal programa. E o nome desse programa é **interpretador**. Aqui, estamos por enquanto usando a palavra interpretador em sentido amplo, como todo o ciclo deste robusto programa que permite executar código na linguagem Python.

Há dois jeitos primordiais de iniciar o interpretador: por meio do modo interativo, que nos dá um prompt onde podemos digitar comandos em Python, apertar ```Enter```, e vê-los sendo executados (ou receber um erro); e por meio de scripts, onde especificamos um arquivo ```.py```, usando o comando ```python3 [arquivo].py```, e o interpretador executa a totalidade daquele arquivo de uma vez. Há diversas opções que podem ser configuradas ao invocar o interpretador pelo terminal para customizar seu comportamento, tal como consultar a versão instalada, obter ajuda, passar argumentos ao código a ser executado, entre outros. Não os abordaremos aqui, mas eles podem ser consultados [na documentação oficial](https://docs.python.org/3/using/cmdline.html#using-on-general).

Ocorre que não existe apenas um único programa que é capaz de executar Python. Existem vários. A cada um desses damos o nome de **implementação**. A primeira e mais importante implementação de Python, usada como principal referência e cujo funcionamento é, por vezes, até mesmo mencionado na documentação oficial da própria linguagem, é escrita em C - ou seja, seu interpretador é um programa compilado a partir de instruções escritas em C. É chamada de **CPython**. Existem outras implementações em outras linguagens, como Jython em Java e PyPy em próprio Python(!), ou apenas para permitir a execução da linguagem em outros ambientes ou com outras arquiteturas. Ao percorrer os deste material, você perceberá que algumas vezes o comportamento da linguagem depende da implementação. Vamos nos basear principalmente em CPython, embora eu tente deixar explícito sempre que algum assunto dependa desta implementação em particular.

No restante do presente tópico, vamos pincelar um pouco do funcionamento interno do interpretador, tentando manter a discussão abstrata o suficiente para se aplicar a qualquer implementação.

## O ciclo REPL

O que chamamos acima de interpretador executa uma série de tarefas necessárias para a linguagem chegar do nível de texto compreendido por seres humanos até o resultado ou efeito de um cálculo computacional no nível da máquina. Ao alimentarmos um trecho de código ao interpretador no modo interativo, ele dispara o próximo passo de um ciclo abstrato chamado de REPL, em inglês: *read*, *eval*, *print*, *loop*. Essa abstração, por vezes chamada de **modelo de execução** é muito útil para entender o funcionamento do interpretador.

Em resumo, podemos dizer que a etapa *read* traduz o texto para algo que a máquina entende, a etapa *eval* então executa de fato essas instruções, a etapa *print* imprime na tela o resultado da computação (se houver), e a etapa *loop* faz o programa continuar executando.

### *Read*: o tempo de compilação

Quando fornecemos ao Python um trecho de código, a primeira coisa que ele faz, na etapa *read*, é chamar um *parser* para transformar o texto em uma estrutura lógica.

Como parte do *parser*, o código primeiramente passa por uma tokenização feita pelo **analisador léxico**, processo que quebra o código-fonte em unidades mínimas de significado chamadas de tokens, enquadrando-as em categorias simbólicas.

In [4]:
abc = 1

No código acima, o analisador léxico reconhece que há três tipos de tokens: um nome, de conteúdo ```abc```; um operador, de conteúdo ```=``` (atribuição); e um valor numérico, de conteúdo $1$.

É o analisador léxico que nos diz como podemos, por exemplo, introduzir espaços, quebrar linhas ou indentar o código. Afinal, esses componentes delimitam elementos mínimos de significado. É ele que reconhece as palavras reservadas (*keywords*), os caracteres de escape, as f-strings, os espaços inúteis ou linhas em branco, e o descarte de comentários, e é também ele quem sabe que ```3.``` é um ```float``` enquanto ```3``` é um ```int```.

Posteriormente, a sequência de tokens é passada ao **analisador sintático**, que é a parte do *parser* capaz de dizer se aquela sequência de tokens, classificados por tipo, obedece ou não à estrutura gramatical da linguagem. Por exemplo, após o comando ```del```, a linguagem espera uma referência a um objeto, mas no caso do operador ```+```, ela espera um objeto não somente depois, mas também antes. Depois de ```def```, espera-se um nome de função, parênteses contendo eventuais argumentos, dois pontos, e um bloco de código, que caso contenha várias linhas lógicas deve ser indentado ou separado por ```;```.

In [ ]:
a = 1 + 2

A expressão acima já é um pouco mais complexa, envolvendo três nomes e dois operadores. Na verdade, na maioria das vezes, o código é muito mais complexo que isso, envolvendo enormes estruturas aninhadas e blocos de controle. A análise sintática dispõe de uma série de técnicas para verificar a corretude de uma sequência de tokens e transformá-la em uma estrutura lógica de onde se pode rastrear a correta ordem de execução do código por parte da máquina. Em CPython especificamente, usa-se a técnica de estruturar o resultado da análise sintática em uma estrutura chamada Árvore Sintática Abstrata (AST), que no caso do exemplo acima seria mais ou menos assim:

In [ ]:
#    =
#   / \
#  a   +
#     / \
#    1   2

Notar que, como pessoas usuárias da linguagem, nós podemos cometer erros tanto léxicos quanto sintáticos. Mas o Python trata todos esses tipos de erro como sintáticos. Nos dois exemplos abaixo, o primeiro erro é léxico (pois não existe token para o caractere ```$``` em Python), enquanto o segundo é sintático (faltando um dois-pontos ```:```), mas o interpretador sempre levanta a exceção ```SyntaxError```:

In [2]:
$

SyntaxError: invalid syntax (1334801704.py, line 1)

In [3]:
if True
    print()

SyntaxError: expected ':' (1297360230.py, line 1)

O último elemento da etapa *read* é o **compilador**. Ele recebe o código que já foi logicamente estruturado e o converte em um objeto do tipo bytecode, um conjunto de instruções que alguma máquina consegue executar, juntamente com tabelas de constantes, nomes e variáveis locais.

Geralmente, as implementações usam uma **máquina virtual** (**VM**) para executar esse código, de modo que ele não é um binário lido pela máquina física, mas sim um conjunto otimizado e compacto de *opcodes* (instruções simples) intermediário, passível de ser executado em tempo de execução, que é o próximo tema deste tópico. Vale notar que existem implementações de Python que o transformam em uma linguagem compilada, a exemplo da implementação Cython. Isso permite que um código-fonte seja transformado em linguagem de máquina propriamente dita (e não um bytecode intermediário), geralmente usando uma linguagem intermediadora como C. Se usado apenas dessa forma, o Python já não é mais um programa completo como discutimos no começo desta seção, deixando para o próprio sistema da máquina executar aquele código. Porém, o uso que se faz muitas vezes desse tipo de técnica é permitir que certos trechos mais computacionalmente intensivos do código sejam pré-compilados e, posteriormente, acessados normalmente pelo interpretador, melhorando o desempenho.

### *Eval*: o tempo de execução

Após passar pela etapa de ler o código em texto e transformá-lo em bytecode, o Python precisa agora executar esse código. Isso é feito por meio de todo um ciclo robusto comumente chamado de **runtime** (tempo de execução).

Já antecipamos acima o primeiro elemento do runtime: a máquina virtual (VM). Neste ponto, lembre-se que usamos até aqui usamos a palavra "interpretador" de maneira ampla, para designar todo o programa que estamos descrevendo, mas essa palavra pode ser usada em sentido mais estrito, para descrever apenas um componente do tempo de execução, também chamado de **interpretador de bytecode**. É na interação entre a VM e o interpretador de bytecode que reside o coração do tempo de execução.

A VM em si não faz nada. Ela é um mero aglomerado de estruturas de dados que descreve a execução e seu estado atual. É composta por uma pilha de **frames** (ou quadros de execução), objetos contendo um bloco específico de código a ser executado (em bytecode) e algumas informações sobre esse bloco (as variáveis ali definidas e seus nomes, os operandos, os blocos de controle tais como loops e tratamentos de exceções, ferramentas para depuração). A pilha de frames também é chamada de *call stack* ou pilha de chamadas, porque via de regra, a chamada a uma função cria um novo frame. A importação de um módulo ou script também cria um frame, entre outras formas especiais de criação de frames. Cada frame é executado em ordem. Quando executado, o frame é retirado da pilha e passa-se ao próximo frame.

Notar que, como funções podem ser chamadas dentro de funções, esse modelo permite recursão. O traceback padrão que recebemos do Python nos mostra onde, na pilha de frames, a execução levantou algum erro:

In [7]:
def fn1():
    raise Exception('Traceback exibe: chamada a fn2 -> chamada a fn1 dentro de fn2 -> erro dentro de fn1')

def fn2(function):
    function()

fn2(fn1)

Exception: Traceback exibe: chamada a fn2 -> chamada a fn1 dentro de fn2 -> erro dentro de fn1

Em CPython especificamente, tudo na VM é implementado por meio de pilhas. Cada frame contém um ponteiro para o objeto compilado pelo compilador (bytecode + nomes e vínculos), uma pilha de operandos (resultados intermediários da computação), e estado da execução (em qual frame e execução estamos, qual é a pilha de chamadas).

Dissemos, no entanto, que a VM em si não faz nada. Quem maneja todas essas estruturas de dados contendo informações, instruções, pilhas etc. é o interpretador de bytecode. O laço principal do interpretador fica consumindo a estrutura de dados da VM e alterando seu estado interno de acordo com as regras da linguagem. Ao ler o bytecode, o interpretador solicita a uma outra interface, o **sistema de runtime**, que acione o sistema operacional para executar as instruções correspondentes. Executadas essas instruções, o interpretador faz as modificações necessárias na VM - e.g., retira um frame da pilha - e segue para as próximas execuções, até terminar a pilha e executar tudo que foi pedido (ou ficando eternamente preso em um loop infinito).

Aqui, chegamos ao último componente do tempo de execução. O sistema de runtime, também chamado de **ambiente de runtime**, faz toda a ponte entre essa dupla VM/interpretador e o sistema operacional. É ele que leva as instruções do interpretador para o sistema. Com isso, permite a execução do programa na máquina.

O runtime gerencia o uso da memória por parte do Python: aloca e desaloca memória, incluindo o manejo do coletor de lixo (*garbage collector*), e dispõe de um espaço chamado *heap* para o armazenamento de objetos criados dinamicamente a fim de satisfazer as necessidades do tempo de execução. Também gerencia os dicionários globais, namespaces e módulos (incluindo a importação de módulos), permite a leitura e a escrita de arquivos, e maneja o sistema de exceções do Python.

Quanto a este último ponto, do que foi exposto até aqui podemos observar que exceções podem ser levantadas em qualquer ponto das etapas *read* e *eval*. A geração de exceções é descentralizada. Mas o Python consegue fazer um manejo centralizado disso, por meio do runtime. Veremos como isso é feito quando tratamos de exceções. **PROMESSA**

### *Print* e *loop*: a continuação do runtime


Quando o fim da etapa de avaliação leva à produção de um resultado, este é retornado pelo interpretador (agora em sentido amplo) e impresso na tela. Isso não altera a execução do programa em si, apenas permite a visualização ou interação dinâmica no terminal.

A etapa de loop pode parecer igualmente óbvia, consistindo no momento em que o terminal está esperando por um novo input. O que é importante aqui é que esta etapa mantém o mesmo estado do runtime até aqui (incluindo o heap e os módulos importados). Os frames e vínculos do módulo principal persistem.

Se você está rodando o código desse texto no Jupyter Notebook, uma sessão interativa de Python está aberta e rodando, de modo que quando rodamos:

In [5]:
import math
a = 2

e, logo em seguida:

In [6]:
print(math.pi * a)

6.283185307179586


, não recebemos um erro. Não abrimos e fechamos o Python ao executar uma célula, pois estamos em modo interativo. ```math``` já foi importado e o nome ```a``` já foi definido ao rodarmos alguma célula anteriormente. Isso aconteceria mesmo se esta célula estivesse abaixo na ordem textual. Rodar outra célula depois continua usando o mesmo runtime, por isso contém a memória do objeto ```a``` e do módulo ```math```.

O mesmo acontece se você fizer isso no Google Colab, exceto que nesse caso o Python é rodado em um servidor distante.

## Desagregando o bytecode

pacote dis

também tem um pacote de compila para bytecode

## Bônus

Uma curiosidade pouco conhecida sobre as regras de indentação de Python é que ela é implementada usando tokens de ```IDENT``` e ```DEDENT``` pelo analisador léxico e depois analisada pelo analisador sintático.

...

In [ ]:
# exemplo indentação zuada dado na documentação







modelo de execução: é apenas a explicação abstrata do que deve fazer o runtime
interpretador CPython: é o executável python3.exe, que dispara tudo isso